In [3]:
import os
import sys
import cv2

import random

import numpy as np

from tqdm import tqdm
import pickle

from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense

from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt

%matplotlib inline

Using TensorFlow backend.
/scratch/huangtin_cs537/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/scratch/huangtin_cs537/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/scratch/huangtin_cs537/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np

In [4]:
train_images = os.listdir('../data/train')
test_images = os.listdir('../data/test')

print(len(train_images))
print(len(test_images))

9000
3000


In [5]:
train_images[0]

'cat.2718.jpg'

In [6]:
test_images_data = []

for image in tqdm(test_images):
    image_data = cv2.imread('../data/test/' + image)

    #convert color from BGR to RGB
    image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)    
    image_data = cv2.resize(image_data, (64, 64))
    
    test_images_data.append(image_data)

100%|██████████| 3000/3000 [00:07<00:00, 413.31it/s]


In [11]:
test_images_data = np.array(test_images_data)
test_images_data.shape

(3000, 64, 64, 3)

In [12]:
train_images_data = []
train_images_labels = []

random.shuffle(train_images)

for image in tqdm(train_images):
    image_data = cv2.imread('../data/train/' + image)
    
    #convert color from BGR to RGB
    image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
    
    image_data = cv2.resize(image_data, (64, 64))

    train_images_data.append(image_data)
    train_images_data.append(image_data[:, ::-1]) #flipped image
    
    if image.startswith('cat'):
#         train_images_labels.append(0)
#         train_images_labels.append(0)
        train_images_labels.append([0, 1, 0])
        train_images_labels.append([0, 1, 0])
    elif image.startswith('dog'):
#         train_images_labels.append(1)
#         train_images_labels.append(1)
        train_images_labels.append([1, 0, 0])
        train_images_labels.append([1, 0, 0])
    else:
#         train_images_labels.append(2)
#         train_images_labels.append(2)  
        train_images_labels.append([0, 0, 1])
        train_images_labels.append([0, 0, 1])

100%|██████████| 9000/9000 [00:21<00:00, 413.37it/s]


In [13]:
train_images_data = np.array(train_images_data)
train_images_labels = np.array(train_images_labels)

train_images_data = train_images_data.reshape([-1, 64, 64, 3])
train_images_labels.shape

(18000, 3)

In [ ]:
pickle.dump(train_images_data, open('colored_train_images_data_.pck', 'wb'))

In [ ]:
pickle.dump(test_images_data, open('model_data/colored_test_images_data.pck', 'wb'))

In [ ]:
pickle.dump(train_images_labels, open('model_data/colored_train_images_labels.pck', 'wb'))

In [2]:
file = open('model_data/colored_test_images_data.pck','rb')
test_images_data = pickle.load(file)

In [3]:
file = open('model_data/colored_train_images_data.pck','rb')
train_images_data = pickle.load(file)

In [4]:
file = open('model_data/colored_train_images_labels.pck','rb')
train_images_labels = pickle.load(file)

In [15]:
model_name = 'colored_baseline_adam_dt9000' #

In [16]:
model = Sequential() 
model.add(Conv2D(64, (5, 5), input_shape=(64, 64, 3))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1)) 
  
model.add(Conv2D(32, (3, 3))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(0.1)) 
  
model.add(Conv2D(32, (3, 3))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(0.1)) 
  
model.add(Flatten()) 
model.add(Dense(50)) #Increase to see how accuracy performs 
#consider also with 0.1 dropout it went from training with 25 to training with 45

model.add(Activation('relu')) 
model.add(Dropout(0.1)) #compare 0.5 with to 0.1 then 0.2 and 0.3
model.add(Dense(1)) 
# model.add(Activation('sigmoid')) 
model.add(Dense(3, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy']) 

tensorboard = TensorBoard(log_dir="logs/{}".format(model_name))
# early_stop = EarlyStopping(monitor='val_loss', patience=5)
checkpoint = ModelCheckpoint(filepath='models/' + model_name + '.h5', monitor='val_accuracy', 
                             save_best_only=True)

model.fit(train_images_data, train_images_labels, epochs=100, validation_split=0.2, batch_size=128,
          callbacks=[tensorboard, checkpoint]) #move batch size to 32, 64, 128
#and finally 256 see how accuracy performs

#remember to try out with dropout per layer

Train on 14400 samples, validate on 3600 samples


Epoch 1/100
14400/14400 [==============================] - 27s 2ms/step - loss: 1.1888 - acc: 0.5666 - val_loss: 0.6636 - val_acc: 0.6309
Epoch 2/100


/scratch/huangtin_cs537/miniconda3/envs/myenv/lib/python3.6/site-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with val_accuracy available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


14400/14400 [==============================] - 26s 2ms/step - loss: 0.6638 - acc: 0.6282 - val_loss: 0.6469 - val_acc: 0.6570
Epoch 3/100
14400/14400 [==============================] - 26s 2ms/step - loss: 0.6434 - acc: 0.6460 - val_loss: 0.6522 - val_acc: 0.6344
Epoch 4/100
14400/14400 [==============================] - 27s 2ms/step - loss: 0.6254 - acc: 0.6576 - val_loss: 0.6109 - val_acc: 0.6731
Epoch 5/100
14400/14400 [==============================] - 27s 2ms/step - loss: 0.6095 - acc: 0.6659 - val_loss: 0.6136 - val_acc: 0.6563
Epoch 6/100
14400/14400 [==============================] - 27s 2ms/step - loss: 0.5907 - acc: 0.6809 - val_loss: 0.5790 - val_acc: 0.6952
Epoch 7/100
14400/14400 [==============================] - 28s 2ms/step - loss: 0.5634 - acc: 0.6980 - val_loss: 0.5456 - val_acc: 0.7098
Epoch 8/100
14400/14400 [==============================] - 32s 2ms/step - loss: 0.5292 - acc: 0.7135 - val_loss: 0.5076 - val_acc: 0.7452
Epoch 9/100
14400/14400 [=====================

In [ ]:
model.save('models/' + model_name + '.h5')